##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Creación de un estimador a partir de un modelo Keras

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/tutorials/estimator/keras_model_to_estimator"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a> </td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/tutorials/estimator/keras_model_to_estimator.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a> </td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/tutorials/estimator/keras_model_to_estimator.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fuente en GitHub</a> </td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/tutorials/estimator/keras_model_to_estimator.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar notebook</a> </td>
</table>

> Advertencia: Los estimadores no se recomiendan para código nuevo. Los estimadores ejecutan el código de estilo `v1.Session` que es más difícil de escribir correctamente y que puede tener un comportamiento inesperado; particularmente, cuando se combina con código TF 2. Los estimadores están incluidos dentro de nuestras [garantías de compatibilidad](https://tensorflow.org/guide/versions), pero no se les harán correcciones a menos que se trate de vulneraciones a la seguridad. Para más detalles, consulte la [Guía de migración](https://tensorflow.org/guide/migrate).

## Descripción general

Los estimadores de TensorFlow se pueden aplicar en TensorFlow y se pueden crear a partir de modelos de código `tf.keras` nuevos o existentes. Este tutorial contiene un ejemplo mínimo, completo de ese proceso.

Nota: si tiene un modelo Keras, puede usarlo directamente con [estrategias `tf.distribute`](https://tensorflow.org/guide/migrate/guide/distributed_training) sin convertir a un estimador. En tal caso, ya no se recomienda `model_to_estimator`.

## Preparar

In [ ]:
import tensorflow as tf

import numpy as np
import tensorflow_datasets as tfds

### Creación de un modelo Keras simple.

En Keras, se ensamblan *capas* para construir *modelos*. Un modelo es, por lo general, un gráfico de capas. El tipo más común de modelo es el de capas apiladas: el modelo `tf.keras.Sequential`.

Para crear una red simple y completamente conectada (es decir, un perceptrón multicapa):

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(16, activation='relu', input_shape=(4,)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(3)
])

Compile el modelo y obtenga una versión resumida.

In [ ]:
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              optimizer='adam')
model.summary()

### Creación de una función de entrada

Use la [API de datasets](../../guide/data.md) para escalar a conjuntos de datos más grandes o a entrenamiento con múltiples dispositivos.

Los estimadores deben controlar cuándo y cómo se crea la canalización de entrada. Para que esto sea posible, requieren de una "función de entrada" o `input_fn`. El `Estimator` llamará a esta función sin argumentos. El `input_fn` debe devolver un `tf.data.Dataset`.

In [ ]:
def input_fn():
  split = tfds.Split.TRAIN
  dataset = tfds.load('iris', split=split, as_supervised=True)
  dataset = dataset.map(lambda features, labels: ({'dense_input':features}, labels))
  dataset = dataset.batch(32).repeat()
  return dataset

Pruebe su `input_fn`

In [ ]:
for features_batch, labels_batch in input_fn().take(1):
  print(features_batch)
  print(labels_batch)

### Creación de un estimador a partir del modelo tf.keras.

Un `tf.keras.Model` se puede entrenar con la API de `tf.estimator` al convertir el modelo en un objeto `tf.estimator.Estimator` con `tf.keras.estimator.model_to_estimator`.

In [ ]:
import tempfile
model_dir = tempfile.mkdtemp()
keras_estimator = tf.keras.estimator.model_to_estimator(
    keras_model=model, model_dir=model_dir)

Entrene y evalúe el estimador.

In [ ]:
keras_estimator.train(input_fn=input_fn, steps=500)
eval_result = keras_estimator.evaluate(input_fn=input_fn, steps=10)
print('Eval result: {}'.format(eval_result))